In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import os
import re
import nltk
import random
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, LineTokenizer, RegexpTokenizer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

print(os.listdir("/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/"))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
stopwords_indonesia = stopwords.words('indonesian')
stop_words = set(stopwords.words('indonesian'))
import sys
import os

['Indonesian Query Answering Dataset for Online Essay Test System.zip', 'dict.json', 'Preprocessing', 'Analysis Data', 'Text Preprocessing', 'Data', 'dataset.py', '__pycache__']


In [7]:
pip install nlp_id

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import re
import random
import pandas as pd
import torch
import tensorflow as tf
import numpy as np

from nlp_id.lemmatizer import Lemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import f1_score, cohen_kappa_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from nltk.corpus import stopwords

In [26]:
!cp /content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/dataset.py
!cp /content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/dict.json
import json
import random
import itertools
from google.colab import files

py_file_location = "/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/dict.json"

with open(py_file_location, 'r') as file:
  data_json = json.load(file)

cp: missing destination file operand after '/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/dataset.py'
Try 'cp --help' for more information.
cp: missing destination file operand after '/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/dict.json'
Try 'cp --help' for more information.


In [39]:
# fungsi untuk menghapus kata negasi
def hapus_kata_negasi(kalimat):
    # membuat daftar kata negasi
    negasi = ['tidak', 'bukan', 'belum', 'tak', 'jangan', 'tidaklah', 'bukannya', 'tiada', 'gak', 'ngga', 'nggak', 'enggak']
    # melakukan split kalimat menjadi token
    result = []
    negate = False
    for word in kalimat:
        if any(neg == word for neg in negasi):
            negate = True
        elif negate and word not in string.punctuation:
            word = 'tidak_' + word
            negate = False
        result.append(word)
    return result

def remove_stopword(token):
    return ' '. join(token)

def augment_data(text, dictionary):
    new_sentences = []
    word_synonyms = []
    base_word = []
    words = text.split()
    for i in range(min(5, len(words))):
        synonym = []
        if words[i] in dictionary:
            synonyms = dictionary[words[i]]['sinonim'][0]
            synonym.append(synonyms)
            synonym.append(words[i]) 
            word_synonyms.append(synonym)
        else:
          synonym.append(words[i])
          word_synonyms.append(synonym)
    
    for j in range(5, len(words)):
      a = []
      a.append(words[j])
      word_synonyms.append(a)

    new_sentences = list(itertools.product(*word_synonyms))
    new_sentences = [' '.join(pasangan) for pasangan in new_sentences]
    return new_sentences

In [51]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
punctuation = r'[^\w\s]'

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def qwk_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return cohen_kappa_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def evaluate(dataloader_val, device, model):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

def train_eval(df_final, pretrainedmodel):
    df_final_new = []

    for i, row in df_final.iterrows():
        text = row['jawaban']
        augmented_data = augment_data(text, data_json)
        for j in range(0, len(augmented_data) - 1):
            row['jawaban'] = augmented_data[j]
            new_row = row
            df_final = df_final.append(new_row)
  
    df_final.reset_index(drop=True, inplace=True)
    df_final['nilai'] = pd.qcut(df_final['nilai'], 5, labels=False)

    # make sure that the training set and test set ratio is 80:20
    add = len(df_final[df_final['tipe'] == 'test']) - (round(0.2*(len(df_final[df_final['tipe'] == 'train'])+len(df_final[df_final['tipe'] == 'test']))))
    for i in df_final[df_final['tipe'] == 'test'].sample(n = add).itertuples():
        df_final.at[i.Index, 'tipe'] = 'train'

    # load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(pretrainedmodel, ignore_mismatched_sizes=True)

    encoded_data_train = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='train']['jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='test']['jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df_final[df_final.tipe=='train'].nilai.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df_final[df_final.tipe=='test'].nilai.values)

    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

    model = BertForSequenceClassification.from_pretrained(pretrainedmodel,
                                                          num_labels=5,
                                                          output_attentions=False,
                                                          output_hidden_states=False, ignore_mismatched_sizes=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    batch_size = 4

    dataloader_train = DataLoader(dataset_train,
                                  sampler=RandomSampler(dataset_train),
                                  batch_size=batch_size)

    dataloader_validation = DataLoader(dataset_val,
                                       sampler=SequentialSampler(dataset_val),
                                       batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(),
                      lr=2e-5,
                      eps=1e-8)

    epochs = 4

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=len(dataloader_train)*epochs)

    for epoch in tqdm(range(1, epochs+1)):

        model.train()

        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            outputs = model(**inputs)

            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


        torch.save(model.state_dict(), f'/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data/Model_Save/finetuned_BERT_{df_final["path"]}_epoch_{epoch}.model')

        tqdm.write(f'\nEpoch {epoch}')

        loss_train_avg = loss_train_total/len(dataloader_train)
        tqdm.write(f'Training loss: {loss_train_avg}')

        val_loss, predictions, true_vals = evaluate(dataloader_validation, device, model)
        val_f1 = f1_score_func(predictions, true_vals)
        val_qwk = qwk_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'QWK Score: {val_qwk}')

In [54]:
def asag_systems(path_dir):
    list_pre_trained_model = ['indobenchmark/indobert-lite-base-p2']
    list_dir = os.listdir(path_dir)
    for m in list_pre_trained_model:
        print(m)
        for idx, ele in enumerate(list_dir):
            df_raw = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                                  sheet_name='Soal',
                                  header=1,
                                  index_col=0,
                                  usecols='B:D')

            list_final = []

            for i in df_raw.itertuples():
                list_final.append(
                    {
                        'jawaban': i[1],
                        'nilai': 100,
                        'tipe': 'train',
                        'path': f' augmentasi data',
                    }
                )
                df_tmp = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                                        sheet_name='No.'+str(i.Index),
                                        header=1,
                                        index_col=0,
                                        usecols='B:N')
                
                df_tmp = df_tmp.dropna()
                for j in df_tmp.itertuples():
                    list_final.append(
                        {
                            'jawaban': j[1],
                            'nilai': j[12],
                            'tipe': 'test',
                            'path': f'augmentasi data',
                        }
                    )
            if idx == 0:
                df_final = pd.DataFrame(list_final)
            else:
                df_final.append(pd.DataFrame(list_final), ignore_index=True)

            print(' '.join(ele.rstrip('.xslx').split('_')))
    
            train_eval(df_final, m)


In [ ]:
asag_systems('/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data/Data Lagi')

indobenchmark/indobert-lite-base-p2
Analisis Essay Grading Lifestyle


<ipython-input-51-8d76b8aa0705>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(new_row)
<ipython-input-51-8d76b8aa0705>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(new_row)
<ipython-input-51-8d76b8aa0705>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(new_row)
<ipython-input-51-8d76b8aa0705>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(new_row)
<ipython-input-51-8d76b8aa0705>:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  d

RuntimeError: ignored

In [11]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
punctuation = r'[^\w\s]'

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def qwk_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return cohen_kappa_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def evaluate(dataloader_val, device, model):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

def train_eval(df_final, pretrainedmodel):
    # bin nilai (continuous variable) into intervals
    df_final['nilai'] = pd.qcut(df_final['nilai'], 5, labels=False)

    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: x.lower())
    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: re.sub(punctuation, '', x))
    df_final['jawaban'] = df_final['jawaban'].apply(nltk.word_tokenize)
    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: [word for word in x if word not in stop_words])
    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: remove_stopword(x))
    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: hapus_kata_negasi(x))
    df_final['jawaban'] = df_final['jawaban'].apply(lambda x: remove_stopword(x))

    # make sure that the training set and test set ratio is 80:20
    add = len(df_final[df_final['tipe'] == 'test']) - (round(0.2*(len(df_final[df_final['tipe'] == 'train'])+len(df_final[df_final['tipe'] == 'test']))))
    for i in df_final[df_final['tipe'] == 'test'].sample(n = add).itertuples():
        df_final.at[i.Index, 'tipe'] = 'train'

    # load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(pretrainedmodel, ignore_mismatched_sizes=True)

    encoded_data_train = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='train']['jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='test']['jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df_final[df_final.tipe=='train'].nilai.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df_final[df_final.tipe=='test'].nilai.values)

    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

    model = BertForSequenceClassification.from_pretrained(pretrainedmodel,
                                                          num_labels=5,
                                                          output_attentions=False,
                                                          output_hidden_states=False, ignore_mismatched_sizes=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    batch_size = 15

    dataloader_train = DataLoader(dataset_train,
                                  sampler=RandomSampler(dataset_train),
                                  batch_size=batch_size)

    dataloader_validation = DataLoader(dataset_val,
                                       sampler=SequentialSampler(dataset_val),
                                       batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(),
                      lr=1e-5,
                      eps=1e-8)

    epochs = 10

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=len(dataloader_train)*epochs)

    for epoch in tqdm(range(1, epochs+1)):

        model.train()

        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            outputs = model(**inputs)

            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


        torch.save(model.state_dict(), f'/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data/Model_Save/finetuned_BERT_{df_final["path"]}_epoch_{epoch}.model')

        tqdm.write(f'\nEpoch {epoch}')

        loss_train_avg = loss_train_total/len(dataloader_train)
        tqdm.write(f'Training loss: {loss_train_avg}')

        val_loss, predictions, true_vals = evaluate(dataloader_validation, device, model)
        val_f1 = f1_score_func(predictions, true_vals)
        val_qwk = qwk_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'QWK Score: {val_qwk}')

In [12]:
def asag_systems(path_dir):
    list_pre_trained_model = ['indobenchmark/indobert-lite-base-p2']
    list_dir = os.listdir(path_dir)
    for m in list_pre_trained_model:
        print(m)
        for idx, ele in enumerate(list_dir):
            df_raw = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                                  sheet_name='Soal',
                                  header=1,
                                  index_col=0,
                                  usecols='B:D')

            list_final = []

            for i in df_raw.itertuples():
                list_final.append(
                    {
                        'jawaban': i[1],
                        'nilai': 100,
                        'tipe': 'train',
                        'path': f'{ele}, Negation',
                    }
                )
                df_tmp = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                                        sheet_name='No.'+str(i.Index),
                                        header=1,
                                        index_col=0,
                                        usecols='B:N')
                
                df_tmp = df_tmp.dropna()
                for j in df_tmp.itertuples():
                    list_final.append(
                        {
                            'jawaban': j[1],
                            'nilai': j[12],
                            'tipe': 'test',
                            'path': f'{ele}, Negation',
                        }
                    )
            if idx == 0:
                df_final = pd.DataFrame(list_final)
            else:
                df_final.append(pd.DataFrame(list_final), ignore_index=True)

            print(' '.join(ele.rstrip('.xslx').split('_')))
    
            train_eval(df_final, m)


In [13]:
asag_systems('/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data')

indobenchmark/indobert-lite-base-p2
Analisis Essay Grading Lifestyle


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at indobenchmark/indobert-lite-base-p2 were not used when initializing BertForSequenceClassification: ['pooler.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'pooler.weight', 'encoder.albert_layer_groups.0.albert_layers.0.attention.dense.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.key.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight', 'encoder.albert_layer_groups.0.albert_layers.0.ffn_output.weight', 'encoder.albert_layer_groups.0.albert

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-780573c4e306>", line 1, in <module>
    asag_systems('/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data')
  File "<ipython-input-12-7a6323408cd4>", line 47, in asag_systems
    train_eval(df_final, m)
  File "<ipython-input-11-0337c7d0e96b>", line 164, in train_eval
    outputs = model(**inputs)
  File "/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/transformers/models/bert/modeling_bert.py", line 1562, in forward
    outputs = self.bert(
  File "/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/transform

KeyboardInterrupt: ignored